$ \frac{d\dot{m}}{dt} = \frac{A_1}{L_C}(\phi (N(t), \dot{m})P_1 - P_P(t)) $

$ \frac{d P_P}{dt} = \frac{C_1^2}{\nu _P}(\dot{m}(t) - \alpha (t) K_\nu \sqrt{P_P - P_{out}}) $

$ \begin{matrix} A_1 & = & 2.6\centerdot 10^-3 m² \\
\nu _P & = & 2.0 m³ \\
L_C & = & 2.0 m \\
K_\nu & = & \frac{0.38 kg}{(kPa)^{0.5}s} \\
P_1 & = & 4.5 Bar \\
P{out} & = & 5.0 Bar \end{matrix}
$

$ \frac{d\dot{m}}{dt} = \frac{2.6\centerdot 10^{-3}}{2.0}(1.5\centerdot 4.5 - P_P) $

$ \frac{d P_P}{dt} = \frac{479.029^2}{2.0}(\dot{m} - \alpha {0.38} \sqrt{P_P - 5.0}) $

In [59]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from scipy.optimize import fsolve
import casadi as ca
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [60]:
np.random.seed(42)
A1 = (2.6)*(10**-3)
Lc = 2
kv = 0.38
P1 = 4.5
P_out = 5
C = 479

timestep = 5
epochs = 750
units = 100
nData = 500

alphas = np.random.uniform(0.45,0.55, timestep*2)

def fun(variables, A1, Lc, kv, P1, P_out, C) :
    (x,y) = variables
    eqn_1 = (A1/Lc)* ((1.5 * P1) - y)
    eqn_2 = (C**2)/2 * (x - alphas[0] * kv * np.sqrt(y - P_out))
    return [eqn_1, eqn_2]

result = fsolve(fun, (0, 10), args = (A1, Lc, kv, P1, P_out, C)) 

init_m = result[0]
init_p = result[1]
interval = [np.linspace(i * nData, (i + 1) * nData, nData) for i in range(timestep*2)]
x = ca.MX.sym('x', 2)
alpha = ca.MX.sym('alpha', 1)
massFlowrate = []
PlenumPressure = []
alpha_values = []
RNN_train = []
RNN_trainFut = []
RNN_test = []

for i in range(0,timestep*2): 
    alpha_values.append([np.full(nData, alphas[i])])

    rhs = ca.vertcat((A1/Lc)*((1.5 * P1) - x[1]), (C**2)/2 * (x[0] - alpha * kv * np.sqrt(x[1] - P_out)))
    ode = {'x' : x, 'ode' : rhs, 'p' : alpha }

    F = ca.integrator('F','idas', ode, interval[i][0], interval[i])
    
    sol = F(x0 = [init_m, init_p], p = alphas[i])

    xf_values = np.array(sol["xf"])

    aux1, aux2 = xf_values
    massFlowrate.append(aux1)
    PlenumPressure.append(aux2)
    init_m = aux1[-1]
    init_p = aux2[-1]

    if i < timestep:
        RNN_train.append([aux1, aux2, np.full(nData,alphas[i])])
        RNN_trainFut.append([aux1, aux2, np.full(nData,alphas[i+1])])
    else:
        RNN_test.append([aux1, aux2, np.full(nData,alphas[i])])


In [65]:
fig = make_subplots(rows=1, cols=3, subplot_titles=("Mass Flow Rate vs Time", "Plenum Pressure vs Time", "Alpha vs Time"))

# Adiciona os gráficos de mass flow rate
for i in range(0, timestep * 2):
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(massFlowrate[i]), mode='lines',
                             name='Mass Flow Rate', legendgroup='massflow', showlegend=i == 0), row = 1, col = 1)
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(PlenumPressure[i]), mode='lines',
                             name='Plenum Pressure', legendgroup='pressure', showlegend=i == 0), row = 1, col = 2)
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(alpha_values[i]), mode='lines', 
                             name='Alpha Values', line=dict(dash='dash'), legendgroup='alpha', showlegend=i == 0), row = 1, col = 3)

# Atualiza layout
fig.update_layout(
    title='Mass Flow Rate vs Time',
    xaxis_title='Time',
    yaxis_title='Value',
    grid=dict(rows=1, columns=3),
    template='plotly',
)

# Mostra a figura
fig.show()


In [62]:
RNN_train = np.array(RNN_train)
RNN_trainFut = np.array(RNN_trainFut)
RNN_test = np.array(RNN_test)

X_train = []
y_trainM = []
y_trainP = []
x_test = []

for j in range(len(alphas)):
    for i in range(len(RNN_train[0][0])):
        if j == 0:
            X_train.append(RNN_train[:,:,i])
            x_test.append(RNN_test[:,:,i])
            if i == (len(RNN_train[0][0]) - 1):
                print("test")
                y_trainM.append(RNN_trainFut[:, :1, i])
                y_trainP.append(RNN_trainFut[:, 1:2, i])
            else:
                y_trainM.append(RNN_train[:,:1,i+1])
                y_trainP.append(RNN_train[:,1:2,i+1])

X_train = np.array(X_train)
X_train = X_train/np.max(X_train)

y_trainM = np.array(y_trainM)
y_trainM = y_trainM/np.max(y_trainM)

y_trainP = np.array(y_trainP)
y_trainP = y_trainP/np.max(y_trainP)

x_test = np.array(x_test)
x_testN = x_test/np.max(x_test)


test


In [63]:
# Definindo a entrada
input_layer = Input(shape=(timestep, 3))  # 3 variáveis de entrada

# Camada RNN
rnn_output = LSTM(units, return_sequences=True)(input_layer)

# Saídas para cada variável de saída
output1 = Dense(1,activation='relu', name='Mass')(rnn_output)  # Primeira saída
output2 = Dense(1,activation='relu', name='Press')(rnn_output)  # Segunda saída

# Modelo
model = Model(inputs=input_layer, outputs=[output1, output2])

model.compile(optimizer='adam', 
              loss={'Mass': 'mse', 
                    'Press': 'mse'},
              metrics={'Mass': 'mse', 
                       'Press': 'mse'})

history = model.fit(X_train, [y_trainM, y_trainP], epochs=epochs, verbose = 0)


In [64]:
mass, pressure = np.array(model.predict(x_testN))
max_mass = np.max(x_test[:,:,0])
max_pressure = np.max(x_test[:,:,1])

# Cria uma figura com subplots
fig = make_subplots(rows=1, cols=3, subplot_titles=("Mass Flow Rate vs Time", "Plenum Pressure vs Time"))

# Gráfico da Mass Flow Rate
for i in range(timestep, timestep * 2):
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(mass[:, i - timestep] * max_mass), mode='lines', name=f'Mass Flow Rate {i+1}', line=dict(dash='solid')),
                  row=1, col=1)
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(massFlowrate[i]), mode='lines', name='Model Mass Flow Rate', line=dict(dash='dash', color='red')),
                  row=1, col=1)

# Gráfico da Plenum Pressure
for i in range(timestep, timestep * 2):
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(pressure[:, i - timestep] * max_pressure), mode='lines', name=f'Plenum Pressure {i+1}', line=dict(dash='solid')),
                  row=1, col=2)
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(PlenumPressure[i]), mode='lines', name= 'Model Plenum Pressure', line=dict(dash='dash', color='red')),
                  row=1, col=2)

# Atualiza layout
fig.update_layout(
    title='Mass Flow Rate and Plenum Pressure Over Time',
    xaxis_title='Time',
    yaxis_title='Value',
    template='plotly',
)

# Mostra a figura
fig.show()

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
